## Okex API exploration - Trading

In [3]:
from okex import *
import pandas as pd
from datetime import datetime

- API key, secret and passphrase should be modified in the config.py file (in okex module)  
- My trading bot in Okex is currently being used.

### II. Trading 

In [135]:
account = AccountAPI(API_KEY, API_SECRET, PASSPHRASE, flag='1')
trader = TradeAPI(API_KEY, API_SECRET, PASSPHRASE, flag='1')

We will be placing/canceling orders SPOT BUY/SELL & FUTURES LONG/SHORT.

#### 1. SPOT BUY

In [81]:
account.set_position_mode('net_mode')
trader.place_order(instId = 'BCH-USDT', tdMode = 'cash', side='buy', ordType='market', sz=131)

{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '482327467064176640',
   'sCode': '0',
   'sMsg': '',
   'tag': ''}],
 'msg': ''}

#### 2. SPOT SELL

In [97]:
account.set_position_mode('net_mode')
trader.place_order(instId = 'BCH-USDT', tdMode = 'cash', side='sell', ordType='market', sz=1)

{'code': '1',
 'data': [{'clOrdId': '',
   'ordId': '',
   'sCode': '51008',
   'sMsg': 'Delegate failed. Insufficient BCH balance in account ',
   'tag': ''}],
 'msg': 'Operation failed.'}

#### 3. Futures long

Position mode and leverage configuration

In [174]:
account.set_position_mode('long_short_mode')
account.set_leverage(lever = 20, instId='BCH-USDT-SWAP', mgnMode = 'isolated', posSide='long')

{'code': '0',
 'data': [{'instId': 'BCH-USDT-SWAP',
   'lever': '20',
   'mgnMode': 'isolated',
   'posSide': 'long'}],
 'msg': ''}

In [177]:
# size in target (sum_currency * lever * 100 )
# Example trade 1BCH with a lever of 3 need size = 1 * 3 * 100 = 300
trader.place_order(instId = 'BCH-USDT-SWAP', tdMode = 'isolated', side='buy', ordType='market', sz=2000, posSide='long')

{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '482434253087715328',
   'sCode': '0',
   'sMsg': '',
   'tag': ''}],
 'msg': ''}

#### 4. Futures short

Position mode and leverage configuration

In [136]:
account.set_position_mode('long_short_mode')
account.set_leverage(lever = 7, instId='BCH-USDT-SWAP', mgnMode = 'isolated', posSide='short')

{'code': '0',
 'data': [{'instId': 'BCH-USDT-SWAP',
   'lever': '7',
   'mgnMode': 'isolated',
   'posSide': 'short'}],
 'msg': ''}

In [142]:
# size in target (currency * lever * 100 )
# Example trade 1BCH with a lever of 7 need size = 1 * 7 * 100 = 700
trader.place_order(instId = 'BCH-USDT-SWAP', tdMode = 'isolated', side='sell', ordType='market', sz=700, posSide='short')

{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '482354590348881920',
   'sCode': '0',
   'sMsg': '',
   'tag': ''}],
 'msg': ''}

#### 5. Orders details

- Get an order details

In [87]:
def get_order(trader, instId, ordId):
    order = trader.get_order(instId, ordId)
    if len(order['data'])==0:
        return pd.DataFrame(columns = ['ordId', 'uTime', 'instId', 'state', 'tdMode', 'side', 'posSide', 'lever', 'ordType', 'px', 'fillPx', 'sz', 'fillSz', 'fee', 'uTime'])
    df = pd.DataFrame([x.values() for x in order['data']], columns = order['data'][0].keys())
    df['uTime'] = df['uTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df['cTime'] = df['cTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df = df[['ordId', 'uTime', 'instId', 'state', 'tdMode', 'side', 'posSide', 'lever', 'ordType', 'px', 'fillPx', 'sz', 'fillSz', 'fee', 'uTime']]
    return df

In [89]:
get_order(trader, 'BCH-USDT-SWAP', '482332899086315520')

,ordId,uTime,instId,state,tdMode,side,posSide,lever,ordType,px,fillPx,sz,fillSz,fee,uTime
0,482332899086315520,2022-08-23 15:38:44.457,BCH-USDT-SWAP,filled,isolated,sell,short,7,market,,131.43,700,700,-0.460005,2022-08-23 15:38:44.457


- Get pending orders

In [93]:
def get_orders_pending(trader):
    order = trader.get_orders()
    if len(order['data'])==0:
        return pd.DataFrame(columns = ['ordId', 'uTime', 'instId', 'state', 'tdMode', 'side', 'posSide', 'lever', 'ordType', 'px', 'fillPx', 'sz', 'fillSz', 'fee', 'uTime'])
    df = pd.DataFrame([x.values() for x in order['data']], columns = order['data'][0].keys())
    df['uTime'] = df['uTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df['cTime'] = df['cTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df = df[['ordId', 'uTime', 'instId', 'state', 'tdMode', 'side', 'posSide', 'lever', 'ordType', 'px', 'fillPx', 'sz', 'fillSz', 'fee', 'uTime']]
    return df

In [94]:
get_orders_pending(trader)

,ordId,uTime,instId,state,tdMode,side,posSide,lever,ordType,px,fillPx,sz,fillSz,fee,uTime
0,482346831154917376,2022-08-23 16:34:06.118,BCH-USDT-SWAP,live,isolated,buy,long,20,limit,131,,300,0,0,2022-08-23 16:34:06.118


- Get order history

In [57]:
def get_orders_history(trader):
    order = trader.orders_history_archive('SWAP')
    df = pd.DataFrame([x.values() for x in order['data']], columns = order['data'][0].keys())
    df['uTime'] = df['uTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df['cTime'] = df['cTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    df = df[['ordId', 'uTime', 'instId', 'state', 'tdMode', 'side', 'posSide', 'lever', 'ordType', 'px', 'fillPx', 'sz', 'fillSz', 'pnl', 'fee', 'tpTriggerPx', 'tpTriggerPxType', 'tpOrdPx', 'slTriggerPx', 'slTriggerPxType', 'slOrdPx', 'uTime']]
    return df

In [99]:
get_orders_history(trader).head()

,ordId,uTime,instId,state,tdMode,side,posSide,lever,ordType,px,...,fillSz,pnl,fee,tpTriggerPx,tpTriggerPxType,tpOrdPx,slTriggerPx,slTriggerPxType,slOrdPx,uTime
0,482347312069619712,2022-08-23 16:36:00.805,BCH-USDT-SWAP,filled,isolated,buy,short,7.0,market,,...,1400,-14,-0.92701,,,,,,,2022-08-23 16:36:00.805
1,482332899086315520,2022-08-23 15:38:44.470,BCH-USDT-SWAP,filled,isolated,sell,short,7.0,market,,...,700,0,-0.460005,,,,,,,2022-08-23 15:38:44.470
2,482331317074538496,2022-08-23 15:32:27.309,BCH-USDT-SWAP,filled,isolated,sell,short,7.0,market,,...,700,0,-0.460005,,,,,,,2022-08-23 15:32:27.309
3,482331013327237120,2022-08-23 15:31:14.868,BCH-USDT-SWAP,filled,isolated,sell,long,3.0,market,,...,400,0.1,-0.26314,,,,,,,2022-08-23 15:31:14.868
4,482330988824113176,2022-08-23 15:31:09.035,BCH-USDT-SWAP,filled,isolated,buy,long,3.0,market,,...,300,0,-0.19743,,,,,,,2022-08-23 15:31:09.035


#### 6. Cancel an order

In [71]:
def cancel_order(trader, instId, ordId):
    trader.cancel_order(instId, ordId)

In [95]:
cancel_order(trader, 'BCH-USDT-SWAP', '482346831154917376')

#### 7. Close positions

In [122]:
trader.close_positions(instId='BCH-USDT-SWAP', mgnMode='isolated', posSide='short', ccy=None)

{'code': '0',
 'data': [{'clientId': '',
   'instId': 'BCH-USDT-SWAP',
   'posSide': 'short',
   'tag': ''}],
 'msg': ''}

#### 8. Take Profit/Stop Loss orders

Create TP/SL order

In [152]:
# size in target (currency * lever * 100 )
# Example trade 1BCH with a lever of 7 need size = 1 * 7 * 100 = 700
trader.place_algo_order(instId = 'BCH-USDT-SWAP', tdMode = 'isolated', side='sell', ordType='oco',  sz=700, posSide='short', tpTriggerPx=140, tpOrdPx=140, slTriggerPx=131, slOrdPx=131)

{'code': '0',
 'data': [{'algoId': '482355795477278720', 'sCode': '0', 'sMsg': ''}],
 'msg': ''}

Cancel TP/SL order

In [173]:
trader.cancel_algo_order([{'algoId':'482355795477278720', 'instId': 'BCH-USDT-SWAP'}])

{'code': '0',
 'data': [{'algoId': '482355795477278720', 'sCode': '0', 'sMsg': ''}],
 'msg': ''}

Get TP/SL pending orders

In [159]:
def get_pending_algo_orders(trader):
    orders = trader.order_algos_list('oco')
    if len(orders['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame([x.values() for x in orders['data']], columns = orders['data'][0].keys())
    df['cTime'] = df['cTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df

In [160]:
get_pending_algo_orders(trader)

,activePx,actualPx,actualSide,actualSz,algoId,cTime,callbackRatio,callbackSpread,ccy,instId,...,tag,tdMode,tgtCcy,timeInterval,tpOrdPx,tpTriggerPx,tpTriggerPxType,triggerPx,triggerPxType,triggerTime
0,,,,0,482355795477278720,2022-08-23 17:09:43.379,,,,BCH-USDT-SWAP,...,,isolated,,,140,140,last,,,


Get TP/SL historical orders

In [171]:
def get_history_algo_orders(trader, state='canceled'):
    orders = trader.order_algos_history('oco', state)
    if len(orders['data'])==0:
        return pd.DataFrame()
    df = pd.DataFrame([x.values() for x in orders['data']], columns = orders['data'][0].keys())
    df['cTime'] = df['cTime'].astype('int64').map(lambda x: pd.to_datetime(x, unit='ms'))
    return df

In [172]:
get_history_algo_orders(trader)

,activePx,actualPx,actualSide,actualSz,algoId,cTime,callbackRatio,callbackSpread,ccy,instId,...,tag,tdMode,tgtCcy,timeInterval,tpOrdPx,tpTriggerPx,tpTriggerPxType,triggerPx,triggerPxType,triggerTime
0,,,,0,482355015080882188,2022-08-23 17:06:37.318,,,,BCH-USDT-SWAP,...,,isolated,,,140,140,last,,,
1,,,,0,482354758917959686,2022-08-23 17:05:36.244,,,,BCH-USDT-SWAP,...,,isolated,,,140,140,last,,,
2,,,,0,482354277319585792,2022-08-23 17:03:41.422,,,,BCH-USDT-SWAP,...,,isolated,,,131,131,last,,,
3,,,,0,482353887635189760,2022-08-23 17:02:08.514,,,,BCH-USDT-SWAP,...,,isolated,,,131,131,last,,,
4,,,,0,482353590762352652,2022-08-23 17:00:57.734,,,,BCH-USDT-SWAP,...,,isolated,,,131,131,last,,,
5,,,,0,482353513926897664,2022-08-23 17:00:39.415,,,,BCH-USDT-SWAP,...,,isolated,,,131,131,last,,,
6,,,,0,482353413699809280,2022-08-23 17:00:15.519,,,,BCH-USDT-SWAP,...,,isolated,,,131,131,last,,,
7,,,,0,482342274689544192,2022-08-23 16:15:59.772,,,,BCH-USDT-SWAP,...,,isolated,,,130,130,last,,,
